<a href="https://colab.research.google.com/github/DrAlexSanz/nlpv2-course/blob/master/Article_spinner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download the data and import everything

In [2]:
import numpy as np
import pandas as pd
import textwrap
import nltk
from nltk import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer

In [3]:
!wget -nc https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv

File ‘bbc_text_cls.csv’ already there; not retrieving.



In [4]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
df = pd.read_csv("bbc_text_cls.csv")
df.head()

,text,labels
0,Ad sales boost Time Warner profit\n\nQuarterly...,business
1,Dollar gains on Greenspan speech\n\nThe dollar...,business
2,Yukos unit buyer faces loan claim\n\nThe owner...,business
3,High fuel prices hit BA's profits\n\nBritish A...,business
4,Pernod takeover talk lifts Domecq\n\nShares in...,business


In [10]:
# Select only the business category at the beginning. Remove filter later

texts = df[df["labels"] == "business"]["text"] # I only need the text
texts.head()

0    Ad sales boost Time Warner profit\n\nQuarterly...
1    Dollar gains on Greenspan speech\n\nThe dollar...
2    Yukos unit buyer faces loan claim\n\nThe owner...
3    High fuel prices hit BA's profits\n\nBritish A...
4    Pernod takeover talk lifts Domecq\n\nShares in...
Name: text, dtype: object

In [19]:
# Counts of terms

probs = {} # Key: (w(t-1), w(t+1), value: {w(t): count(w(t))})
# probs will be a dictionary of dictionaries

for doc in texts:
    lines = doc.split("\n") # For each article, divide it in lines
    for line in lines:
        tokens = word_tokenize(line) # Tokenize each line
        for i in range(len(tokens) - 2):# For each word, see the next two words. Save w(t-1) and w(t+1)
            t_0 = tokens[i]
            t_1 = tokens[i + 1]
            t_2 = tokens[i + 2]
            key = (t_0, t_2)
            if key not in probs:
              probs[key] = {} # If the two words don't exist as a pair, initialize it as empty dict.
            # Add count for middle word
            if t_1 not in probs[key]:
                probs[key][t_1] = 1 # First time set it to 1, otherwise just add 1
            else:
                probs[key][t_1] += 1

In [22]:
# Normalize probabilities, go from counts to probabilities

for key, dist in probs.items():# Each key represents a distribution
    total = sum(dist.values())
    for k, v in dist.items():
        dist[k] = v/total

In [ ]:
probs

In [24]:
texts.iloc[0].split("\n")

['Ad sales boost Time Warner profit',
 '',
 'Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.',
 '',
 'The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.',
 '',
 "Time Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free to TimeWarner internet customers a

In [25]:
# Create a function that does the spinning. Do one paragraph at a time, although one sentence at a time also works

def spin_document(doc):
    """
    Take a document, split it into lines, spin it and stitch it back together
    """

    lines = doc.split("\n")
    output = []
    for line in lines:
        if line:
            new_line = line
        else:
            new_line = spin_line(line)
        
        output.append(new_line)

    return "\n".join(output)


In [26]:
detokenizer = TreebankWordDetokenizer()